# Veri Hazırlama

In [1]:
# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

Amacımız, 9 `.csv` dosyasının tamamını tek bir sözlüğe (dict) `data` adında 9 `pandas.DataFrame` olarak yüklemektir. Burada:
- her **anahtar**, csv dosyasının **temizlenmiş adıdır**
- her **değer**, csv dosyasından oluşturulan **DataFrame**tir

```python
data = { 
    'sellers': DataFrame1,
    'orders': DataFrame2,
    ...
    }
```

### 1. `csv_path` değişkenini oluşturun, bu değişken `"csv" klasörü`nün yolunu bir string olarak depolayacaktır

Yolları kolayca yönetmek için Python'ın `pathlib` modülünü kullanalım. Pathlib çok kullanışlı bir `Path` sınıfı tanımlar.

Bir `Path` örneğini yolumuzu tanımlayan bir string vererek oluşturabiliriz.

In [2]:
from pathlib import Path
csv_path = Path("~/.workintech/olist/data/csv").expanduser()
csv_path

PosixPath('/home/zeynepdil/.workintech/olist/data/csv')

_`PosixPath`_? Bu, yolun Posix formatında olduğu anlamına gelir. POSIX (Portable Operating System Interface), Unix benzeri işletim sistemleri için uyumluluk ve birlikte çalışabilirliği sağlamak amacıyla standartlaştırılmış API'ler ve kuralların bir setidir. Hem Linux (örneğin WSL'deki Ubuntu) hem de macOS, POSIX uyumludur.

`expanduser()` kullanarak `~` işaretini ana klasörümüzün gerçek mutlak yoluna genişletiyoruz.

Şimdi klasörümüzdeki dosyaları listelemek için `iterdir()` metodunu kullanabiliriz:

In [3]:
file_paths = list(csv_path.iterdir())
file_paths

[PosixPath('/home/zeynepdil/.workintech/olist/data/csv/olist_geolocation_dataset.csv'),
 PosixPath('/home/zeynepdil/.workintech/olist/data/csv/olist_order_reviews_dataset.csv'),
 PosixPath('/home/zeynepdil/.workintech/olist/data/csv/olist_orders_dataset.csv'),
 PosixPath('/home/zeynepdil/.workintech/olist/data/csv/olist_products_dataset.csv'),
 PosixPath('/home/zeynepdil/.workintech/olist/data/csv/olist_sellers_dataset.csv'),
 PosixPath('/home/zeynepdil/.workintech/olist/data/csv/product_category_name_translation.csv'),
 PosixPath('/home/zeynepdil/.workintech/olist/data/csv/olist_customers_dataset.csv'),
 PosixPath('/home/zeynepdil/.workintech/olist/data/csv/olist_order_payments_dataset.csv'),
 PosixPath('/home/zeynepdil/.workintech/olist/data/csv/olist_order_items_dataset.csv')]

In [4]:
# Kodunuzu aşağıda test edin. Dizindeki ilk csv dosyasını yüklemeyi deneyin
import pandas as pd
pd.read_csv(file_paths[0]).head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


### 2. csv dizinindeki tüm csv dosya adlarını içeren `file_names` listesini oluşturun

- Bunun böyle görünmesi gerekir: `file_names = ['olist_sellers_dataset.csv', ....]`
- `file_paths` adresinden başlayabilirsiniz
- Bir `Path` nesnesi `name` adında bir özelliğe sahiptir

In [5]:

file_names = [file.name for file in file_paths]


print(file_names)


['olist_geolocation_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_orders_dataset.csv', 'olist_products_dataset.csv', 'olist_sellers_dataset.csv', 'product_category_name_translation.csv', 'olist_customers_dataset.csv', 'olist_order_payments_dataset.csv', 'olist_order_items_dataset.csv']


### 3. Sözlük anahtarları listesi `key_names` oluşturun
file_names adresinden başlayarak:
- ".csv" sonekini (varsa) kaldırın
- "_dataset.csv" sonekini (varsa) kaldırın
- "olist_" önekini (varsa) kaldırın

<details>
    <summary>- İpucu - </summary>

- `.replace()`
    
- `str` diziler [ ] ile dilimlenebilecek yinelenebilir nesnelerdir
</details>

In [20]:
key_names = [name.replace("olist_", "").replace("_dataset.csv", "").replace(".csv", "") for name in file_names]
print(key_names)

['geolocation', 'order_reviews', 'orders', 'products', 'sellers', 'product_category_name_translation', 'customers', 'order_payments', 'order_items']


### 4. `data` sözlüğünü oluşturun

```python
data = { 
    'sellers': DataFrame1,
    'orders': DataFrame2,
    'order_items': DataFrame3,
    ...
    }
```
Burada `DataFrame1`, `DataFrame2`, ... gerçek `pandas.DataFrame` nesneleri olmalıdır.

<details>
    <summary>▸ İpucu</summary>

`zip()` metodu iki liste üzerinde yinelemek için çok kullanışlıdır
```python
for (x, y) in zip(['a','b','c'], [1,2,3]):
    print(x,y)

# ('a', 1), ('b', 2), ('c', 3) döndürür
    
```
</details>

In [21]:
data = {}
for key, file in zip(key_names, file_paths):
    data[key] = pd.read_csv(file)
data.keys()
#data['orders'].head()


#data

dict_keys(['geolocation', 'order_reviews', 'orders', 'products', 'sellers', 'product_category_name_translation', 'customers', 'order_payments', 'order_items'])

### 5. `olist/data.py` dosyasındaki `get_data()` metodunu uygulayın

Mantığımızı not defterinden `.py` dosyalarımıza taşımanın zamanı geldi. Bu, bu modül boyunca oluşturacağımız yeni not defterlerinde verileri kolayca yüklememize izin verecek.

Bu modülün klasöründe (bu not defterinin iki seviye yukarısında) `olist/data.py` dosyasını açın ve bu not defterine yazdığınız kodu `get_data()` metoduna taşımaya başlayın. Kodumuzu test etmek için yazdığımız satırları atlayın (`.py` dosyasında listeleri ve DataFrame'leri göstermeye gerek yok).

İşlev çağrıldığında DataFrames içeren sözlüğü döndürmelidir. Bunu csv dosyalarından biriyle deneyip görebilirsiniz:

In [24]:
from olist.data import Olist
olist = Olist()
olist.ping()

data = olist.get_data()
data.keys()

#Olist().get_data()['sellers'].head()

ModuleNotFoundError: No module named 'olist'

### Kodunuzu test edin

In [ ]:
from nbresult import ChallengeResult
from olist.data import Olist
data = Olist().get_data()
result = ChallengeResult('get_data',
    keys_len=len(data),
    keys=sorted(list(data.keys())),
    columns=sorted(list(data['sellers'].columns)),
    vars_used=Olist.get_data.__code__.co_names
    )
result.write()
print(result.check())

❓Bu kod parçası makinenizde her yerden çalışmalı, sadece bu not defterinden değil.
- Yeni bir terminal açın
- Ana klasörünüze gidin `cd`
- Bir `ipython` oturumu başlatın
- Yukarıdaki iki satır kodu test edin 👆

🏁 Tebrikler!

💾 Şunları commit ve push etmeyi unutmayın: 
* bu `data_preparation.ipynb` not defteri ve challenge klasöründeki `tests/get_data.pickle` test sonuçları
* ayrıca `/olist` klasöründeki `data.py` dosyası